#### NLTK Set up

Follow [1.2 Getting Started with NLTK](https://www.nltk.org/book/ch01.html)

A better reference: [NLP with Python](https://bagustris.github.io/nlp-python/index.html)

In [1]:
# Run once, or as many times as you need to configure which files you want access to. 
import nltk
from nltk.util import ngrams
# nltk.download()

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
import pickle

In [3]:
# Carga the CESS_ESP corpus. Vamos a entranar nuestro modelo
training_corpus = nltk.corpus.cess_esp.words()

In [4]:
# Hagas los N-grams, por N=[2,3,4,5]
N=[2,3,4,5]
N_grams={
    1:[],
    2:[],
    3:[],
    4:[],
    5:[]
}

# Tecnicamente, no necesitamos declarar las listas
N_grams_cnt={}
N_grams_freq={}

for n in tqdm(N):
    N_grams[n]=list(
        ngrams(
            training_corpus,
            n
        )
    )
    N_grams_cnt[n]=Counter(N_grams[n])
    N_grams_cnt[f"{n}_total_cnt"]=sum(N_grams_cnt[n].values())
    N_grams_freq[n]={
        ngram: count / N_grams_cnt[f"{n}_total_cnt"] for ngram, count in N_grams_cnt[n].items()
    }
    

100%|██████████| 4/4 [00:11<00:00,  2.89s/it]


In [5]:
N_gram_mle={}
N_gram_mle_str={}

for n in tqdm(N):
    N_gram_cache={} # no es el mismo cache del paper
    N_gram_mle[n]={} # para gana los 
    N_gram_mle_str[n]={}
    for gram in N_grams[n]:
        if gram[0] in N_gram_cache.keys():
            tkn_n_gram_cnt=N_gram_cache[gram[0]]
        else:
            grams_with_matching_first_word = [index for index, tup in enumerate(N_grams[n]) if gram[0]==tup[0]] # Gana los n-grams que lo tiene la primera palabra misma.
            tkn_n_gram_cnt = N_gram_cache[gram[0]] = len(grams_with_matching_first_word) # contar los iguales
        gram_str=" "
        gram_str=gram_str.join(gram)
        if n == 1:
            N_gram_mle[n][gram]=round(N_grams_cnt[n][gram]/N_grams_cnt[f"{n}_total_cnt"], 5)
            N_gram_mle_str[n][gram_str]=round(N_grams_cnt[n][gram]/N_grams_cnt[f"{n}_total_cnt"], 5)
        else:
            N_gram_mle[n][gram]=round(N_grams_cnt[n][gram]/tkn_n_gram_cnt, 5) # calcular MLE
            N_gram_mle_str[n][gram_str]=round(N_grams_cnt[n][gram]/tkn_n_gram_cnt, 5) # calcular MLE

100%|██████████| 4/4 [12:54<00:00, 193.59s/it]


In [8]:
with open("../outputs/n_gram_models.pkl", "wb") as f:
    pickle.dump(N_gram_mle_str, f)